In [139]:
import os
import sys
import time
import json
import re
import csv
import datetime as dt
import multiprocessing as mp
import pandas as pd
import numpy as np
import sqlalchemy as sql
from lxml import etree as ET
from collections import OrderedDict

## Scraping the NSF
Use BeautifulSoup to crawl the NSF site and extract awards, award recipients (PI's), univerities and NSF internal divisions. Dump'em into JSON/CSV for analysis/warehousing...
<br>TODO: use multiprocessing pool to speed up crawls!

Notes from Ivan:
Use Lxml rather than b4
don't fund before use
Classes!
Declarative

In [226]:
def main():
    # remove stuff I don't want...
    rep = {"Division ": "", "Of ": "","of ": "", "Div ": "","Directorate ":"","Divn ":"","for ":""}
    rep = dict((re.escape(k), v) for k, v in rep.items())
    pattern = re.compile("|".join(rep.keys()))
    paths = [str(x) for x in range(2000,2017)]

    def scrape(path,MySQL=False):
        # Builder functions
        def awards_builder(tree):
            return awards.append(
                    OrderedDict([
                            ('AwardID',tree.find('Award/AwardID').text),
                            ('AwardTitle',tree.find('Award/AwardTitle').text),
                            ('AwardEffectiveDate',tree.find('Award/AwardEffectiveDate').text),
                            ('AwardExpirationDate',tree.find('Award/AwardExpirationDate').text),
                            ('AwardAmount',tree.find('Award/AwardAmount').text),
                            ('InstitutionName',tree.find('Award/Institution/Name').text),
                            ('Division',pattern.sub(lambda m: rep[re.escape(m.group(0))],
                                                    tree.find('Award/Organization/Division')[0].text)),
                            ('EmailAddress',tree.find('Award/Investigator/EmailAddress').text)
                    ]))

        def abstracts_builder(tree):
            return abstracts.append(
                    OrderedDict([
                            ('AwardID',tree.find('Award/AwardID').text),
                            ('Abstract',tree.find('Award/AbstractNarration').text)
                     ]))

        def institutions_builder(tree):
            return institutions.append(
                    OrderedDict([
                            ('InstitutionName',tree.find('Award/Institution/Name').text),
                            ('StreetAddress',tree.find('Award/Institution/StreetAddress').text),
                            ('CityName',tree.find('Award/Institution/CityName').text),
                            ('StateCode',tree.find('Award/Institution/StateCode').text),
                            ('ZipCode',tree.find('Award/Institution/ZipCode').text),
                            ('CountryName',tree.find('Award/Institution/CountryName').text)
                    ]))
        # what if two pis, what if pi moves universities?
        def PI_builder(tree):
            return PI.append(
                    OrderedDict([
                            ('EmailAddress',tree.find('Award/Investigator/EmailAddress').text),
                            ('FirstName',tree.find('Award/Investigator/FirstName').text),
                            ('LastName',tree.find('Award/Investigator/LastName').text)
                    ]))

        def xml_parse(file):
            tree = ET.parse(file)
            try:
                awards_builder(tree)
            except:
                return(tree.find('Award/AwardID').text,"is missing a award key")

            abstracts_builder(tree)

            institutions_builder(tree)

            PI_builder(tree)

        # main script
        print("***",path,"***")
        # iterate through the path directory
        [xml_parse(os.path.join('data/'+path,file)) for file in os.listdir('data/'+path)]
        return

    def json_dump():
        dict_NSF = [awards,
                    abstracts,
                    institutions,
                    PI]

        outfiles = ["NSF_AWARDS.json",
                    "NSF_ABSTRACTS.json",
                    "NSF_INSTITUTIONS.json",
                    "NSF_PI.json"]

        for i in range(len(dict_NSF)):
            with open(outfiles[i], 'w') as outfile:
                json.dump(dict_NSF[i], outfile, indent=4)
    def csv_dump():
        dict_NSF = [awards,
                    abstracts,
                    institutions,
                    PI]

        outfiles = ["NSF_AWARDS.csv",
                    "NSF_ABSTRACTS.csv",
                    "NSF_INSTITUTIONS.csv",
                    "NSF_PI.csv"]

        for _dict,file in zip(dict_NSF,outfiles):
            with open(file, 'w') as output_file:
                dict_writer = csv.DictWriter(output_file, _dict[0].keys())
                dict_writer.writeheader()
                dict_writer.writerows(_dict)

    def remove_duplicates(dict_list):
        return [dict(tupleized) for tupleized in set(tuple(item.items()) for item in dict_list)]

    # Create global lists.
    awards = []
    abstracts = []
    institutions = []
    PI = []
    # scrape the files
    start = time.time()
    for path in paths:
        scrape(path)
    print(len(awards),"records")
    print("Scraped in", round(time.time()-start, 2), "sec.")
    print("****************")
    # remove doops
    start = time.time()
    awards = remove_duplicates(awards)
    PI = remove_duplicates(PI)
    institutions = remove_duplicates(institutions)
    print("Duplicates removed in",round(time.time()-start,2), "sec.")
    # dump'em
    start = time.time()
    csv_dump()
    print("Dumped into CSV in",round(time.time()-start,2), "sec.")
main()

*** 2000 ***
*** 2001 ***
*** 2002 ***
*** 2003 ***
*** 2004 ***
*** 2005 ***
*** 2006 ***
*** 2007 ***
*** 2008 ***
*** 2009 ***
*** 2010 ***
*** 2011 ***
*** 2012 ***
*** 2013 ***
*** 2014 ***
*** 2015 ***
*** 2016 ***
185345 records
Scraped in 278.76 sec.
****************
Duplicates removed in 11.1 sec.
Dumped into CSV in 27.18 sec.


In [224]:
def SQLite_dump(infile,Type):
    print("*********************************************************************")
    # open SQlite connection
    disk_engine = sql.create_engine('sqlite:///NSF.db')
    start = dt.datetime.now()
    tableName = infile.replace("NSF_","").replace(".csv","").replace(".json","")
    if(Type=='csv'):
        chunksize = 20000
        j = 0
        # Break each CSV into chunks to avoid inmemory stoarge of dataset.
        for df in pd.read_csv(infile, chunksize=chunksize, iterator=True, encoding='utf-8'):
            if(tableName=='AWARDS'):
                df['AwardEffectiveDate']  = pd.to_datetime(df['AwardEffectiveDate'])
                df['AwardExpirationDate'] = pd.to_datetime(df['AwardExpirationDate'])
            j+=1
            print("{} seconds: {} records dumped into SQLite table {}".format((dt.datetime.now() - start).seconds,j*chunksize,tableName))
            if(j==1):
                df.to_sql(tableName,disk_engine,if_exists='replace')
            else:
                df.to_sql(tableName,disk_engine,if_exists='append')
            
    elif(Type=='json'):
        df = pd.read_json(infile)
        if(tableName=='AWARDS'):
            df['AwardEffectiveDate']  = pd.to_datetime(df['AwardEffectiveDate'])
            df['AwardExpirationDate'] = pd.to_datetime(df['AwardExpirationDate'])
        print("{} seconds: {} records dumped into SQLite table {}".format((dt.datetime.now() - start).seconds,len(df),tableName))
        df.to_sql(tableName,disk_engine,if_exists='replace')

In [225]:
outfilesCSV  = ["NSF_AWARDS.csv",
                "NSF_ABSTRACTS.csv",
                "NSF_INSTITUTIONS.csv",
                "NSF_PI.csv"]
for infile in outfilesCSV:
    SQLite_dump(infile,Type='csv')

*********************************************************************
4 seconds: 15000 records dumped into SQLite table AWARDS
9 seconds: 30000 records dumped into SQLite table AWARDS
14 seconds: 45000 records dumped into SQLite table AWARDS
20 seconds: 60000 records dumped into SQLite table AWARDS
27 seconds: 75000 records dumped into SQLite table AWARDS
36 seconds: 90000 records dumped into SQLite table AWARDS
49 seconds: 105000 records dumped into SQLite table AWARDS
55 seconds: 120000 records dumped into SQLite table AWARDS
61 seconds: 135000 records dumped into SQLite table AWARDS
67 seconds: 150000 records dumped into SQLite table AWARDS
73 seconds: 165000 records dumped into SQLite table AWARDS
78 seconds: 180000 records dumped into SQLite table AWARDS
81 seconds: 195000 records dumped into SQLite table AWARDS
*********************************************************************
0 seconds: 15000 records dumped into SQLite table ABSTRACTS
21 seconds: 30000 records dumped into SQL

## Analysis of NSF 2015 
Cool things to look at
<br> Common words in titles
<br> email each PI with a status update.
<br> Chloropleth about instituional funding.

In [249]:
#DYNAMIC QUERRYING
def sql_Querry(fromTable,db='NSF',fromCol=-1,WHERE=-1,LIMIT=-1,colsToDisplay='*'):
    disk_engine = sql.create_engine('sqlite:///'+db+'.db') #open connection to SQLite
    if(fromCol==-1 and LIMIT==-1):                 # general search
        print("search case 1")
        querry = 'SELECT {} FROM {}'.format(colsToDisplay.replace('[',"").replace(']',""),fromTable)
    elif(fromCol==-1 and LIMIT!=-1): # general search with limit
        print("search case 2")
        querry = 'SELECT {} FROM {} LIMIT {}'.format(colsToDisplay.replace('[',"").replace(']',""),fromTable,LIMIT)
    elif(fromCol!=-1 and WHERE!=-1 and LIMIT==-1): # search a column for a whereClause
        print("search case 3")
        querry = 'SELECT {} FROM {} WHERE {}="{}" COLLATE NOCASE'.format(colsToDisplay.replace('[',"").replace(']',""),fromTable,fromCol,WHERE)
    elif(fromCol!=-1 and WHERE!=-1 and LIMIT!=-1): # search a column for a whereClause with a limit
        print("search case 4")
        querry = 'SELECT {} FROM {} WHERE {}="{}" COLLATE NOCASE LIMIT {}'.format(colsToDisplay.replace('[',"").replace(']',""),fromTable,fromCol,WHERE,LIMIT)
    return pd.read_sql_query(querry, disk_engine,index_col='index')
sql_Querry(db='NSF',fromTable='AWARDS',fromCol='InstitutionName',WHERE='University of Washington',LIMIT=5)

search case 4


,AwardID,AwardTitle,AwardEffectiveDate,AwardExpirationDate,AwardAmount,InstitutionName,Division,EmailAddress
index,,,,,,,,
207,567,Workshop on Value-Sensitive Design: Cultivatin...,2000-05-15 00:00:00.000000,2001-04-30 00:00:00.000000,35000,University of Washington,Information & Intelligent Systems,borning@cs.washington.edu
247,717,Collaborative Research: A Sea Test of the PROD...,2000-01-15 00:00:00.000000,2000-12-31 00:00:00.000000,8295,University of Washington,Ocean Sciences,johnson@ocean.washington.edu
257,735,NEP GLOBEC: Mesoscale Euphausiid and Hake Dist...,2000-04-01 00:00:00.000000,2006-09-30 00:00:00.000000,520740,University of Washington,Ocean Sciences,bhickey@u.washington.edu
281,904,SGER: Calibration of FeTi-Oxide Thermobaromete...,2000-05-01 00:00:00.000000,2002-04-30 00:00:00.000000,19885,University of Washington,Earth Sciences,None
314,990,Shipboard Scientific Support Equipment 2000,2000-07-15 00:00:00.000000,2002-06-30 00:00:00.000000,82496,University of Washington,Ocean Sciences,mcduff@ocean.washington.edu


In [248]:
sql_Querry(fromTable='AWARDS',LIMIT=10) # same as df.head(10), but doesn't read all files into memory first.

search case 2


,AwardID,AwardTitle,AwardEffectiveDate,AwardExpirationDate,AwardAmount,InstitutionName,Division,EmailAddress
index,,,,,,,,
0,0,Regulation of Sn-Glycerol-3-Phosphate Metaboli...,1986-07-01 00:00:00.000000,1986-07-01 00:00:00.000000,0,Virginia Polytechnic Institute and State Unive...,Molecular and Cellular Bioscience,tilarson@vt.edu
1,9,Design of Cutting Tools for High Speed Milling,2000-06-15 00:00:00.000000,2004-05-31 00:00:00.000000,280000,University of Florida,"Civil, Mechanical, & Manufact Inn",jziegert@uncc.edu


In [251]:
sql_Querry(fromTable='AWARDS').info()

search case 1
<class 'pandas.core.frame.DataFrame'>
Int64Index: 185345 entries, 0 to 5344
Data columns (total 8 columns):
AwardID                185345 non-null int64
AwardTitle             185342 non-null object
AwardEffectiveDate     185345 non-null object
AwardExpirationDate    185345 non-null object
AwardAmount            185345 non-null int64
InstitutionName        185345 non-null object
Division               185345 non-null object
EmailAddress           179320 non-null object
dtypes: int64(2), object(6)
memory usage: 12.7+ MB


Let's take a look at the columns...

In [ ]:
award_df.info()

For RDBA this works great... However, for publishing results nobody wants columns displayed_like_this! So here's a helper function to convert underscored columns into respectible, well-mannered headers. Appropriated from Stackoverflow user <a target="_blank" href='http://stackoverflow.com/a/6425628/5094480'>Siegfried Gevatter</a>

In [ ]:
def beautify(underscored_word):
    # check for acronym
    if(len(underscored_word)>2):
        return ' '.join(x.capitalize() or '_' for x in underscored_word.split('_'))
    else:
        return underscored_word

In [ ]:
def beautify1(camelCaseWord):
    # check for acronym
    return(''.join(map(lambda x: x if x.islower() else " "+x, camelCaseWord)))

A simple, flexible reporting function to find the top n funded column.

In [252]:
def top_funds(df,col,n=5):
    return df.groupby(col).apply(lambda x:x['AwardAmount'].sum()).sort_values(ascending=False).head(n)

In [ ]:
award_df.groupby('Division').apply(lambda x:x['AwardAmount'].sum()).sort_values(ascending=False).head()

In [253]:
def top_awarded(df,col,n):
    return df['Division'].value_counts().head(n)

In [ ]:
"University of Texas".split()
org_df[org_df['InstitutionName'].str.contains("|".join("University of Texas".split()),case=False)]

In [ ]:
award_df[award_df['InstitutionName'].str.contains('New York',case=False)]

In [ ]:
top_funds(award_df[award_df['InstitutionName'].str.contains('Yale',case=False)],'Division',10)
# if unique names are more than 1, show results for one with greatest hits...

In [ ]:
top_awarded(award_df,'Division',10)

In [ ]:
def report(InstitutionName,Division):
    return award_df[(award_df.InstitutionName==InstitutionName) & 
                    (award_df.Division==Division)].merge(abstract_df,on='AwardID').set_index('AwardID')[
                    ["AwardTitle","EmailAddress","Abstract"]]

report("University of Washington","Ocean Sciences")

In [ ]:
def topics_finder(topic):
    df = pd.merge(left=award_df,right=abstract_df,on='AwardID')
    #df = (df.loc[(df['AwardTitle'].notnull()) & (df['Abstract'].notnull()) &
    #             (df['AwardTitle'].str.contains(topic,case=False)) | # search title
    #              df['Abstract'].str.contains(topic,case=False)])    # search abstract SLOW!
    
    df = (df.loc[(df['AwardTitle'].notnull()) &
                 (df['AwardTitle'].str.contains(topic,case=False))])
    #df = pd.merge(left=df,right=pi_df, on='EmailAddress')            # get PI info
    return(df[['AwardID','AwardTitle','AwardEffectiveDate','EmailAddress']])
topics_finder("omics")

In [ ]:
def PI_Report(PI):
    # search email of full name
    try:
        return award_df.loc[award_df.EmailAddress.isin(pi_df.EmailAddress[pi_df.FullName==PI.lower()].values)]
        #return df.AwardTitle.values
    except:
        return(PI,"Not found in Our database")
    # search awards for email
    # display states
PI_Report('nicole lovenduski')

In [ ]:
award_df[award_df.EmailAddress=='jig@ldeo.columbia.edu']

In [ ]:
# TODO fix PI shared names with email addresses!
def top_PI():
    award_df['PI'] = award_df.first_name+" "+award_df.last_name
    pi_df['PI'] = pi_df.first_name+" "+pi_df.last_name
    top_fund_col = []
    for PI in pi_df.PI:
        top_fund_col.append(
            {'PI':PI, 'funding_division': np.asarray(award_df[award_df.PI==PI].division)[0],"Total_Award_Money" : award_df[award_df['PI']==PI].award_amount.sum()})
    fund_df = pd.DataFrame(sorted(top_fund_col, key=lambda k: k['Total_Award_Money'], reverse=True))
    fund_df = pd.merge(left=fund_df,right=pi_df,on='PI',how='inner')
    fund_df.index = fund_df.index+1
    fund_df.rename(columns=lambda x: beautify(x), inplace=True)
    return fund_df[["Total Award Money","PI","Email Address","Funding Division"]][:25].drop_duplicates()
top_PI()

TODO
Short term:
Make time series of funding
University profile(maybe 
Chunk Pandas for scale?
Use SQLite rather than SQL? (maybe)...
Bokeh interactive plots (or D3.js)

Long term:
PI's who switch schools.